# Assignment 2 - Python for Text Analysis

| Name          | Student ID    | Section contributed | Section Edited | Other Contributions |
| ------------- | ------------- | ------------- | ------------- | -------------|
| Dela Torre, Kyla | 301361824 | Data collection, Reflection | Topic Labelling | Ideas for Project |
| Ferrera, Johann | 301601581 | Data Cleaning, Functions | Topic Labelling | Set Up Github Repo |

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import gensim
from gensim import corpora
from gensim.models import LdaModel

## Data Preprocessing

### Import and Examine Data

In [2]:
df = pd.read_csv('./data/guardian_articles_2022.csv', encoding = 'utf-8')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   article_id          11414 non-null  object
 1   sectionName         11414 non-null  object
 2   webTitle            11414 non-null  object
 3   webUrl              11414 non-null  object
 4   bodyContent         11414 non-null  object
 5   webPublicationDate  11414 non-null  object
 6   id                  11414 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 624.3+ KB


,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,football/2022/jan/31/liverpools-late-push-to-s...,Football,Liverpool’s late push to sign Fabio Carvalho f...,https://www.theguardian.com/football/2022/jan/...,Liverpool were frustrated in their attempt to ...,2022-01-31T23:41:16Z,138340
1,world/2022/jan/31/spike-in-covid-cases-in-engl...,World news,Rise in Covid cases in England as reinfections...,https://www.theguardian.com/world/2022/jan/31/...,"Another 92,000 Covid cases were reported for E...",2022-01-31T23:30:38Z,138341
2,australia-news/2022/feb/01/five-great-reads-au...,Australia news,"Five Great Reads: Australian curlers, an intim...",https://www.theguardian.com/australia-news/202...,"Good morning, welcome to February and to Five ...",2022-01-31T23:30:03Z,138342
3,football/2022/jan/31/barcelona-aubameyang-loan...,Football,Aubameyang to seal Barcelona move from Arsenal...,https://www.theguardian.com/football/2022/jan/...,Pierre-Emerick Aubameyang has passed a medical...,2022-01-31T23:26:50Z,138343
4,world/2022/jan/31/mexico-fourth-journalist-kil...,World news,‘We only have a pen’: fury as fourth journali...,https://www.theguardian.com/world/2022/jan/31/...,Journalists in Mexico have responded with fury...,2022-01-31T23:07:16Z,138344


### Define and Preprocess Text

Based on Week 13's lab.

In [3]:
# define a function that: 1. tokenizes, 2. lowercases and
# 3. lemmatizes

# create the lemmatizer first
lemmatizer = WordNetLemmatizer()

# define the function
def process_text(text):
    clean_text = re.sub(r'<.*?>', '', text) # I don't think there's any HTML tags in here but I'll keep it in here just in case
    words = word_tokenize(clean_text.lower())
    cleaned_words = [word for word in words if word.isalpha() and word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]
    return lemmatized_words

In [4]:
df['processedBody'] = df['bodyContent'].apply(process_text)

In [5]:
df.head()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id,processedBody
0,football/2022/jan/31/liverpools-late-push-to-s...,Football,Liverpool’s late push to sign Fabio Carvalho f...,https://www.theguardian.com/football/2022/jan/...,Liverpool were frustrated in their attempt to ...,2022-01-31T23:41:16Z,138340,"[liverpool, frustrated, attempt, sign, fabio, ..."
1,world/2022/jan/31/spike-in-covid-cases-in-engl...,World news,Rise in Covid cases in England as reinfections...,https://www.theguardian.com/world/2022/jan/31/...,"Another 92,000 Covid cases were reported for E...",2022-01-31T23:30:38Z,138341,"[another, covid, case, reported, england, mond..."
2,australia-news/2022/feb/01/five-great-reads-au...,Australia news,"Five Great Reads: Australian curlers, an intim...",https://www.theguardian.com/australia-news/202...,"Good morning, welcome to February and to Five ...",2022-01-31T23:30:03Z,138342,"[good, morning, welcome, february, five, great..."
3,football/2022/jan/31/barcelona-aubameyang-loan...,Football,Aubameyang to seal Barcelona move from Arsenal...,https://www.theguardian.com/football/2022/jan/...,Pierre-Emerick Aubameyang has passed a medical...,2022-01-31T23:26:50Z,138343,"[aubameyang, passed, medical, due, confirmed, ..."
4,world/2022/jan/31/mexico-fourth-journalist-kil...,World news,‘We only have a pen’: fury as fourth journali...,https://www.theguardian.com/world/2022/jan/31/...,Journalists in Mexico have responded with fury...,2022-01-31T23:07:16Z,138344,"[journalist, mexico, responded, fury, despair,..."


## Data Processing

### Split up into months into separate dataframes

The dataset only has entries from January until June. The following code puts each month into a separate dataset.

In [6]:
# convert the date from the df. You only need to do this once for the df
df['webPublicationDate'] = df['webPublicationDate'].astype(str)

In [7]:
# create df_jan
df_jan = df[df['webPublicationDate'].str.split('-').str[1] == '01']

In [8]:
# create df_feb
df_feb = df[df['webPublicationDate'].str.split('-').str[1] == '02']

In [9]:
# create df_mar
df_mar = df[df['webPublicationDate'].str.split('-').str[1] == '03']

In [10]:
# create df_apr
df_apr = df[df['webPublicationDate'].str.split('-').str[1] == '04']

In [11]:
# create df_may
df_may = df[df['webPublicationDate'].str.split('-').str[1] == '05']

In [12]:
# create df_jun
df_jun = df[df['webPublicationDate'].str.split('-').str[1] == '06']

### Running gensim for each year

#### Dictionary

In [13]:
dictionary_jan = corpora.Dictionary(df_jan['processedBody'])
dictionary_feb = corpora.Dictionary(df_feb['processedBody'])
dictionary_mar = corpora.Dictionary(df_mar['processedBody'])
dictionary_apr = corpora.Dictionary(df_apr['processedBody'])
dictionary_may = corpora.Dictionary(df_may['processedBody'])
dictionary_jun = corpora.Dictionary(df_jun['processedBody'])

#### Corpus

In [14]:
corpus_jan = [dictionary_jan.doc2bow(text) for text in df_jan['processedBody']]
corpus_feb = [dictionary_feb.doc2bow(text) for text in df_feb['processedBody']]
corpus_mar = [dictionary_mar.doc2bow(text) for text in df_mar['processedBody']]
corpus_apr = [dictionary_apr.doc2bow(text) for text in df_apr['processedBody']]
corpus_may = [dictionary_may.doc2bow(text) for text in df_may['processedBody']]
corpus_jun = [dictionary_jun.doc2bow(text) for text in df_jun['processedBody']]

### LDA Model

In [15]:
# create the LDA model
lda_model_jan = LdaModel(corpus_jan, num_topics=10, id2word=dictionary_jan, passes=15)
lda_model_feb = LdaModel(corpus_feb, num_topics=10, id2word=dictionary_feb, passes=15)
lda_model_mar = LdaModel(corpus_mar, num_topics=10, id2word=dictionary_mar, passes=15)
lda_model_apr = LdaModel(corpus_apr, num_topics=10, id2word=dictionary_apr, passes=15)
lda_model_may = LdaModel(corpus_may, num_topics=10, id2word=dictionary_may, passes=15)
lda_model_jun = LdaModel(corpus_jun, num_topics=10, id2word=dictionary_jun, passes=15)

## Results

### Section Topics

These are the topics possible in the entire 2022 dataset:

In [16]:
section_counts = df['sectionName'].value_counts()
filtered_sections = section_counts[section_counts > 50] # Only counts over 50

with pd.option_context('display.max_rows', None):
    print(filtered_sections)

sectionName
World news            1449
Australia news         894
Football               756
Opinion                753
Sport                  702
Business               589
US news                582
UK news                567
Politics               524
Life and style         416
Music                  400
Environment            399
Film                   389
Society                348
Books                  329
Television & radio     328
Stage                  231
Art and design         169
Money                  157
Culture                154
Food                   153
Technology             148
Media                  138
Global development     137
Education              106
News                   105
Travel                  90
Science                 88
Fashion                 63
Law                     58
Name: count, dtype: int64


### Topics

These are the most popular topics from Jan-Jun 2022 from the articles from The Guardian.

**Note**: "u" refers to the US.

#### January 2022

In [17]:
# print the top 20 words for each topic in January of 2022
topics = lda_model_jan.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: said, year, price, data, one, people, u, cost, also, service, uk, may, tax, personal, company, last, government, time, rate, bill

Topic 1: said, nadal, set, open, match, one, tennis, year, australian, slam, time, first, final, two, also, barty, title, grand, would, win

Topic 2: woman, court, first, one, supreme, year, black, justice, also, abortion, song, biden, child, died, time, life, bbc, two, case, music

Topic 3: said, people, would, year, u, say, one, time, also, could, ukraine, country, russia, told, government, day, new, last, right, two

Topic 4: said, report, police, gray, would, investigation, inquiry, met, one, year, say, time, could, people, evidence, also, minister, party, johnson, full

Topic 5: one, like, time, first, game, year, say, get, two, thing, team, back, good, last, make, really, new, world, play, would

Topic 6: said, government, people, year, new, would, health, covid, school, also, pandemic, child, australia, country, need, vaccine, care, time, ca

#### February 2022

In [18]:
# print the top 20 words for each topic in February of 2022
topics = lda_model_feb.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: person, said, one, two, court, time, told, like, game, word, also, round, letter, would, minute, first, add, man, get, another

Topic 1: said, would, energy, agl, plant, year, australia, company, coal, government, last, power, abortion, frog, woman, new, one, also, price, police

Topic 2: say, garden, one, get, fish, wale, year, time, also, plant, would, home, like, scotland, back, good, two, hallos, way, food

Topic 3: game, team, one, said, player, league, time, two, season, year, first, last, would, club, goal, england, back, win, three, city

Topic 4: say, one, like, people, year, time, life, first, new, work, get, would, make, way, show, also, thing, film, world, feel

Topic 5: said, court, woman, year, people, would, say, school, police, child, also, one, u, case, government, time, right, student, group, justice

Topic 6: said, people, government, year, health, covid, would, new, pandemic, also, say, one, time, could, service, uk, home, public, worker, need

Topic 7: sai

#### March 2022

In [19]:
# print the top 20 words for each topic in March of 2022
topics = lda_model_mar.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: say, said, one, like, time, would, people, also, year, two, spa, day, first, make, back, new, flood, water, could, get

Topic 1: one, news, royal, british, bbc, said, u, two, letter, slavery, time, year, william, people, caribbean, jamaica, world, reparation, britain, human

Topic 2: said, year, told, person, would, court, australia, new, people, child, also, family, one, study, time, work, health, two, guardian, could

Topic 3: said, government, would, year, people, uk, minister, also, cost, could, australia, new, price, last, week, energy, company, time, country, tax

Topic 4: people, said, covid, smith, best, oscar, health, one, year, also, film, nh, death, time, award, first, hospital, actor, like, rock

Topic 5: said, people, ppe, war, train, right, party, ukraine, would, city, russian, also, say, u, first, vote, one, place, medpro, year

Topic 6: say, one, like, time, year, people, first, would, world, get, thing, life, game, back, said, two, day, make, also, way

Topic 

#### April 2022

In [20]:
# print the top 20 words for each topic in April of 2022
topics = lda_model_apr.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: one, say, year, like, time, first, said, also, would, new, people, way, work, world, album, box, two, life, music, last

Topic 1: one, person, year, said, people, man, would, also, two, work, soldier, say, court, space, photo, may, time, new, want, life

Topic 2: game, team, player, league, club, season, one, first, said, time, football, would, year, two, point, win, last, back, liverpool, goal

Topic 3: said, mp, woman, would, people, johnson, one, minister, say, party, labour, tory, conservative, like, time, prime, think, also, year, get

Topic 4: said, year, people, would, price, also, new, cost, say, one, time, government, company, month, could, uk, last, need, food, energy

Topic 5: say, people, said, one, year, time, way, also, life, first, day, like, could, get, new, police, would, know, university, friend

Topic 6: twitter, musk, said, company, would, platform, say, make, one, like, also, social, people, free, speech, oil, user, u, medium, share

Topic 7: one, like, ti

#### May 2022

In [21]:
# print the top 20 words for each topic in May of 2022
topics = lda_model_may.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: said, australia, government, new, party, china, minister, would, labor, climate, liberal, pacific, country, election, seat, australian, policy, albanese, national, people

Topic 1: said, people, say, one, year, also, time, would, new, water, like, could, case, uk, get, may, many, community, make, two

Topic 2: trump, republican, said, election, state, would, u, house, also, former, candidate, company, georgia, primary, party, court, vote, biden, president, one

Topic 3: russian, ukraine, said, russia, war, ukrainian, u, country, people, world, city, force, also, putin, would, military, one, woman, invasion, region

Topic 4: season, player, game, one, year, team, league, club, time, first, final, last, said, two, would, say, back, also, like, new

Topic 5: said, people, school, child, year, gun, police, would, u, state, one, government, also, time, shooting, law, home, two, family, right

Topic 6: one, like, say, time, make, year, people, new, also, way, even, thing, day, work,

#### June 2022

In [22]:
# print the top 20 words for each topic in June of 2022
topics = lda_model_jun.print_topics(num_words=20)
for topic_num, topic in topics:
    print(f"Topic {topic_num}: ", end="")
    words = topic.split(' + ')
    word_list = [word.split('*')[1].strip('\"') for word in words] 
    print(", ".join(word_list))
    print()

Topic 0: said, government, would, right, year, people, state, abortion, court, new, also, u, woman, law, say, minister, one, public, decision, uk

Topic 1: say, year, one, price, like, people, said, new, also, market, business, cost, time, u, company, make, product, many, money, crypto

Topic 2: said, people, year, say, australia, also, climate, new, one, home, time, population, area, food, could, energy, would, change, land, housing

Topic 3: film, july, say, year, one, said, first, festival, like, light, work, new, australia, art, also, people, time, artist, tv, world

Topic 4: said, johnson, trump, election, party, would, one, labour, tory, people, committee, conservative, minister, mp, time, vote, could, prime, government, also

Topic 5: said, ukraine, russian, russia, war, would, country, ukrainian, u, nato, defence, force, putin, also, one, military, year, could, city, people

Topic 6: people, one, time, life, child, work, year, like, said, would, first, also, new, way, say, thin

#### Labelled Topics

Below are each topic above labelled:

**January**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | Finance, Economics, Public Policy | said, year, price, data, one, people, u, cost, also, service, uk, may, tax, personal, company, last, government, time, rate, bill |
| 1 | Sports, Tennis | said, nadal, set, open, match, one, tennis, year, australian, slam, time, first, final, two, also, barty, title, grand, would, win |
| 2 | Social Justice, Cultural Issues | woman, court, first, one, supreme, year, black, justice, also, abortion, song, biden, child, died, time, life, bbc, two, case, music |
| 3 | International News, Russia-Ukraine War | said, people, would, year, u, say, one, time, also, could, ukraine, country, russia, told, government, day, new, last, right, two |
| 4 | Revisiting Partygate From Last Year | said, report, police, gray, would, investigation, inquiry, met, one, year, say, time, could, people, evidence, also, minister, party, johnson, full |
| 5 | Sports, Tournament | one, like, time, first, game, year, say, get, two, thing, team, back, good, last, make, really, new, world, play, would |
| 6 | Australian COVID-19 Measures | said, government, people, year, new, would, health, covid, school, also, pandemic, child, australia, country, need, vaccine, care, time, case, one |
| 7 | Sports | one, year, also, would, said, like, new, club, time, work, say, people, first, player, world, could, way, u, make, day |
| 8 | General Life | said, say, one, people, year, test, also, time, like, would, new, first, day, week, two, get, back, going, last, care |
| 9 | Politics | johnson, said, minister, party, would, government, prime, one, mp, people, time, tory, u, could, conservative, also, week, political, former, right |


**February**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | General News | person, said, one, two, court, time, told, like, game, word, also, round, letter, would, minute, first, add, man, get, another |
| 1 | Energy, Environment | said, would, energy, agl, plant, year, australia, company, coal, government, last, power, abortion, frog, woman, new, one, also, price, police |
| 2 | UK Nature | say, garden, one, get, fish, wale, year, time, also, plant, would, home, like, scotland, back, good, two, hallos, way, food |
| 3 | Soccer, Sports | game, team, one, said, player, league, time, two, season, year, first, last, would, club, goal, england, back, win, three, city |
| 4 | Film, Arts & Culture | say, one, like, people, year, time, life, first, new, work, get, would, make, way, show, also, thing, film, world, feel |
| 5 | Social Issues, Women's Rights | said, court, woman, year, people, would, say, school, police, child, also, one, u, case, government, time, right, student, group, justice |
| 6 | COVID-19 | said, people, government, year, health, covid, would, new, pandemic, also, say, one, time, could, service, uk, home, public, worker, need |
| 7 | Russian-Ukraine War | said, ukraine, russian, russia, putin, ukrainian, would, people, country, war, u, government, also, minister, invasion, military, force, president, kyiv, sanction |
| 8 | Energy, Russian-Ukraine Crises | price, gas, russia, said, ukraine, energy, oil, russian, market, bank, could, year, would, u, supply, europe, world, cost, country, crisis |
| 9 | US-UK Relations | said, company, bank, would, uk, russian, year, trump, also, sanction, u, new, business, russia, government, country, one, including, state, former |


**March**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | Lifestyle | say, said, one, like, time, would, people, also, year, two, spa, day, first, make, back, new, flood, water, could, get |
| 1 | Royal Family, Colonialism | one, news, royal, british, bbc, said, u, two, letter, slavery, time, year, william, people, caribbean, jamaica, world, reparation, britain, human |
| 2 | Family-Related Social Issues | said, year, told, person, would, court, australia, new, people, child, also, family, one, study, time, work, health, two, guardian, could |
| 3 | Politics | said, government, would, year, people, uk, minister, also, cost, could, australia, new, price, last, week, energy, company, time, country, tax |
| 4 | COVID-19 and Entertainment | people, said, covid, smith, best, oscar, health, one, year, also, film, nh, death, time, award, first, hospital, actor, like, rock |
| 5 | Politics, Russian-Ukraine | said, people, ppe, war, train, right, party, ukraine, would, city, russian, also, say, u, first, vote, one, place, medpro, year |
| 6 | General Reflection | say, one, like, time, year, people, first, would, world, get, thing, life, game, back, said, two, day, make, also, way |
| 7 | Environment, Climate | year, one, new, climate, also, work, world, first, april, said, say, like, people, u, specie, film, gallop, many, would, could |
| 8 | Russian-Ukraine War | said, russian, ukraine, russia, war, ukrainian, u, putin, trump, would, president, people, one, biden, country, also, force, city, military, state |
| 9 | Education | school, woman, said, child, one, year, family, also, club, say, new, would, two, time, student, people, league, last, education, work |


**April**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | Life, Media | one, say, year, like, time, first, said, also, would, new, people, way, work, world, album, box, two, life, music, last |
| 1 | Society | one, person, year, said, people, man, would, also, two, work, soldier, say, court, space, photo, may, time, new, want, life |
| 2 | Sports, Football/Soccer | game, team, player, league, club, season, one, first, said, time, football, would, year, two, point, win, last, back, liverpool, goal |
| 3 | UK Politics | said, mp, woman, would, people, johnson, one, minister, say, party, labour, tory, conservative, like, time, prime, think, also, year, get |
| 4 | Economics | said, year, people, would, price, also, new, cost, say, one, time, government, company, month, could, uk, last, need, food, energy |
| 5 | Opinion | say, people, said, one, year, time, way, also, life, first, day, like, could, get, new, police, would, know, university, friend |
| 6 | Social Media | twitter, musk, said, company, would, platform, say, make, one, like, also, social, people, free, speech, oil, user, u, medium, share |
| 7 | Education, Art and Film | one, like, time, year, say, people, new, film, first, show, school, world, life, make, work, two, may, feel, way, also |
| 8 | US News, Society | say, like, one, time, year, music, trump, also, people, first, day, said, work, way, life, get, back, new, would, thing |
| 9 | World News, Russian-Ukraine Conflict | said, government, would, ukraine, russian, people, russia, war, also, country, one, u, minister, year, ukrainian, new, uk, right, state, could |


**May**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | Australian Politics | said, australia, government, new, party, china, minister, would, labor, climate, liberal, pacific, country, election, seat, australian, policy, albanese, national, people |
| 1 | UK News, Society | said, people, say, one, year, also, time, would, new, water, like, could, case, uk, get, may, many, community, make, two |
| 2 | US Politics | trump, republican, said, election, state, would, u, house, also, former, candidate, company, georgia, primary, party, court, vote, biden, president, one |
| 3 | World News, Russian-Ukraine Conflict | russian, ukraine, said, russia, war, ukrainian, u, country, people, world, city, force, also, putin, would, military, one, woman, invasion, region |
| 4 | Sports | season, player, game, one, year, team, league, club, time, first, final, last, said, two, would, say, back, also, like, new |
| 5 | US News, Public Safety, Law | said, people, school, child, year, gun, police, would, u, state, one, government, also, time, shooting, law, home, two, family, right |
| 6 | UK News, Culture, Society | one, like, say, time, make, year, people, new, also, way, even, thing, day, work, food, get, queen, garden, may, london |
| 7 | Politics | said, johnson, would, government, minister, energy, price, cost, year, party, tax, company, mp, one, people, prime, report, could, time, also |
| 8 | Film | say, one, like, time, year, life, people, first, woman, get, film, show, would, new, think, work, know, feel, thing, two |
| 9 | Sports, Football/Soccer | liverpool, one, league, madrid, year, final, time, champion, real, like, first, would, team, game, season, city, last, two, way, football |


**June**
| Index          | Topic    | Words     |
| ------------- | ------------- | ------------- |
| 0 | UK Politics, Law | said, government, would, right, year, people, state, abortion, court, new, also, u, woman, law, say, minister, one, public, decision, uk |
| 1 | Business, Economics | say, year, one, price, like, people, said, new, also, market, business, cost, time, u, company, make, product, many, money, crypto |
| 2 | Environment | said, people, year, say, australia, also, climate, new, one, home, time, population, area, food, could, energy, would, change, land, housing |
| 3 | Australian News, Film, Arts, Culture | film, july, say, year, one, said, first, festival, like, light, work, new, australia, art, also, people, time, artist, tv, world |
| 4 | Politics, Elections | said, johnson, trump, election, party, would, one, labour, tory, people, committee, conservative, minister, mp, time, vote, could, prime, government, also |
| 5 | World News, Russian-Ukraine Conflict, Military | said, ukraine, russian, russia, war, would, country, ukrainian, u, nato, defence, force, putin, also, one, military, year, could, city, people |
| 6 | US News | people, one, time, life, child, work, year, like, said, would, first, also, new, way, say, thing, book, vaccine, u, story |
| 7 | Life, Society | people, one, year, like, say, life, said, time, work, also, would, new, first, woman, school, u, family, way, london, make |
| 8 | Sports, Football/Soccer | england, player, game, team, year, one, said, last, time, first, new, day, football, club, world, could, test, two, city, match |
| 9 | Opinion | one, like, year, say, time, first, people, said, two, get, day, back, also, show, new, would, really, world, going, thing |